In [ ]:
import duckdb
import pandas as pd
import pprint

In [ ]:
# Connect to your local DuckDB database file
conn = duckdb.connect('../data/br_funds.db')


# List all tables in the database
tables = conn.execute("SHOW TABLES").fetchall()
print("Available tables:", tables)


# close the connection when done
conn.close()

### EA of fund_semantic_search_view 

In [ ]:
conn = duckdb.connect('../data/br_funds.db')

df = conn.execute("SELECT * FROM fund_semantic_search_view LIMIT 20").fetchdf()

conn.close()

df.head()

In [ ]:
example_search_text = df["searchable_text"].iloc[0]

pprint.pprint(example_search_text)

In [ ]:
example_search_text = df["searchable_text"].iloc[1]

pprint.pprint(example_search_text)

In [ ]:
example_search_text = df["searchable_text"].iloc[2]

pprint.pprint(example_search_text)

In [ ]:
example_search_text = df["searchable_text"].iloc[3]

pprint.pprint(example_search_text)

In [ ]:
example_search_text = df["searchable_text"].iloc[4]

pprint.pprint(example_search_text)

In [ ]:
example_search_text = df["searchable_text"].iloc[5]

pprint.pprint(example_search_text)

### Cross-Lingual Test: English Query → Portuguese Fund Text

Test if OpenAI embeddings can match English queries to Portuguese fund descriptions!

In [ ]:
import os
from openai import OpenAI
import numpy as np

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def get_embedding(text: str, model="text-embedding-3-large"):
    """Get embedding for a text string"""
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

def cosine_similarity(a, b):
    """Calculate cosine similarity between two vectors"""
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
# Get a sample fund with Portuguese text
conn = duckdb.connect('../data/br_funds.db')
sample_fund = conn.execute("""
    SELECT cnpj, legal_name, searchable_text 
    FROM fund_semantic_search_view 
    LIMIT 1
""").fetchdf()
conn.close()

print("📊 Sample Fund (Portuguese):")
print(f"CNPJ: {sample_fund['cnpj'].iloc[0]}")
print(f"Name: {sample_fund['legal_name'].iloc[0]}")
print(f"\nSearchable Text Preview (first 500 chars):")
print(sample_fund['searchable_text'].iloc[0][:500] + "...")

In [ ]:
# Embed the Portuguese fund text
fund_text = sample_fund['searchable_text'].iloc[0]
fund_embedding = get_embedding(fund_text)

print("🔍 Testing Cross-Lingual Semantic Similarity\n")
print("=" * 60)

# Test queries in different languages
test_queries = [
    ("Portuguese query about the fund type", "fundo de investimento em renda fixa"),
    ("English query about fixed income", "fixed income fund of investment"),
    ("English query about bonds", "government bond fund"),
    ("English query about ESG", "sustainable investment ESG"),
    ("Random unrelated query", "technology startup venture capital"),
]

for label, query in test_queries:
    query_embedding = get_embedding(query)
    similarity = cosine_similarity(fund_embedding, query_embedding)
    
    # Color code the similarity score
    if similarity > 0.7:
        emoji = "🟢"
    elif similarity > 0.5:
        emoji = "🟡"
    else:
        emoji = "🔴"
    
    print(f"{emoji} {label}")
    print(f"   Query: '{query}'")
    print(f"   Similarity: {similarity:.3f} ({similarity*100:.1f}%)")
    print()